In [4]:
import pandas as pd
import os
import random
import itertools

In [60]:
class Market:
    def __init__(self, currencies, dataPath, referenceCurrency='USD', 
                 initialValue=1000, transactionFee=0.005, initialTime=None):
        self.path = dataPath
        self.currencies = []
        self.value = initialValue
        self.fee = transactionFee
        self.time = initialTime #Fill in
        self.currencies = (currencies)
        self.currencies.remove('USD')
        self.currencies.insert(0, 'USD')
        self.m = len(self.currencies)
        self.majorPairs = ['EURUSD', 'GBPUSD']
        self.portfolio = [None] * (self.m)
        pairs = list(itertools.permutations(currencies, 2))
        self.df = self.importFile(pairs)
        self.dfProcessed = {}
        print(self.df.keys())
         
    def importFile(self, pairs):
        df = {}
        dataPath = self.path
        availableFiles = os.listdir(dataPath)
        for pairTuple in pairs:
            pair = pairTuple[0] + pairTuple[1]
            if (pair + '.csv') in availableFiles:
                if dataPath.endswith('/'):
                    if os.path.isfile(dataPath + pair + '.csv'):
                        df[pair] = pd.read_csv(dataPath + pair + '.csv', delimiter='\t', 
                                               usecols=['Timestamp', 'Open', 'High', 'Low', 'Close'])
                    else:
                        continue
                else:
                    if os.path.isfile(dataPath + '/' + pair + '.csv'):
                        df[pair] = (pd.read_csv(dataPath + '/' + pair + '.csv', delimiter='\t', 
                                                usecols=['Timestamp', 'Open', 'High', 'Low', 'Close']))
                    else:
                        continue
        return df
    
    def getPortfolioSize(self):
        return self.m
    
    def getPairs(self):
        return self.pairs
    
    def initPortfolio(self):
        index = 0
        while index < len(self.portfolio):
            random.seed(5)
            self.portfolio[index] = random.random()
            index += 1
        sumNum = sum(self.portfolio)
        index = 0
        while index < len(self.portfolio):
            self.portfolio[index] /= sumNum
            self.portfolio[index] = round(self.portfolio[index],2)
            index += 1
        
    def reallocate(self, currencyAllocation):
        exchange = currencyAllocation - self.portfolio
        oldPortfolio = self.portfolio.copy()
        while exchange.count(0.0) != len(exchange):
            minIndex = exchange.index(min(exchange))
            maxIndex = exchange.index(max(exchange))
            exchangeCurrencies(minIndex, maxIndex, min(abs(exchange[minIndex]), exchange[maxIndex]))
            
    def incrementTime(self):
        currentTime = str(self.getCurrentTime())
        currentIndex = self.df[self.majorPairs[0]].loc[self.df[self.majorPairs[0]]['Timestamp'] 
                                                       == currentTime].index.values[0]
        self.time = str(self.df[self.majorPairs[0]]['Timestamp'].loc[currentIndex + 1])
        
    def timestep(self):
        if self.time is None:
            for pair in self.majorPairs:
                if pair in self.df.keys():
                    self.time = self.df[pair]['Timestamp'].loc[0]
        else:
            self.incrementTime()
    
    def getCurrentTime(self):
        return self.time
    
    def calculateValue(self):
        i = 0
        totalValue = self.value
        newValue = 0
        for currency in self.currencies:
            exchangeRate = getRate(currency, 'USD')
            if currency == 'USD':
                exchangeRate = 1
            newValue += totalValue * self.portfolio[i] * exchangeRate
            i += 1
        
    def exchangeCurrencies(self, source, target, amount):
        exchangeRate, found = getRate(source, target)
        self.portfolio[source] -= amount
        self.portfolio[target] += amount
        if found:
            self.value -= amount * self.fee * self.value
        if not found:
            self.value -= amount * 2 * self.fee * self.value
    
    def getRate(self, source, target):
        pair = source + target
        if pair in self.df.keys():
            return self.df[pair].loc[self.df[pair]['Timestamp'] == getCurrentTime()]['Close'], True
        else:
            pair = target+source
            if pair in elf.df.keys():
                return self.df[pair].loc[self.df[pair]['Timestamp'] == getCurrentTime()]['Close'], True
            else:
                return self.df[source + 'USD'].loc[self.df[source + 'USD']['Timestamp'] 
                                                   == getCurrentTime()]['Close'], False
    def processFrames(self):
        for key, frame in self.df.items():
            currentFrame = frame.copy()
            #currentFrame.loc[0, ('Open', 'High', 'Low', 'Close')] = 1.0
            print(currentFrame[:10])
            print(currentFrame['Open'].loc[:10].pct_change())
#             for i in range(1, len(currentFrame.index)):
#                 currentFrame.loc[i, 'Open'] = currentFrame.loc[i, 'Open'] / currentFrame.loc[i - 1, 'Open']
#                 currentFrame.loc[i, 'High'] = currentFrame.loc[i, 'High'] / currentFrame.loc[i - 1, 'High']
#                 currentFrame.loc[i, 'Low'] = currentFrame.loc[i, 'Low'] / currentFrame.loc[i - 1, 'Low']
#                 currentFrame.loc[i, 'Close'] = currentFrame.loc[i, 'Close'] / currentFrame.loc[i - 1, 'Close']

In [61]:
#Test
test = Market(['EUR','USD'], os.path.abspath('../Data_Processing/ProcessedData'))
test.processFrames()

dict_keys(['EURUSD'])
         Timestamp     Open     High      Low    Close
0  20170102 020000  1.05155  1.05197  1.05155  1.05190
1  20170102 020100  1.05209  1.05209  1.05177  1.05179
2  20170102 020200  1.05177  1.05198  1.05177  1.05178
3  20170102 020300  1.05188  1.05200  1.05188  1.05200
4  20170102 020400  1.05196  1.05204  1.05196  1.05203
5  20170102 020600  1.05196  1.05204  1.05196  1.05204
6  20170102 020700  1.05205  1.05210  1.05205  1.05209
7  20170102 020800  1.05210  1.05210  1.05209  1.05209
8  20170102 020900  1.05208  1.05209  1.05208  1.05209
9  20170102 021000  1.05208  1.05211  1.05207  1.05209
0          NaN
1     0.000514
2    -0.000304
3     0.000105
4     0.000076
5     0.000000
6     0.000086
7     0.000048
8    -0.000019
9     0.000000
10    0.000038
Name: Open, dtype: float64
